In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import csv

In [ ]:
def file_builder(links,brand):
    """Функция  записывает построчно  в файл спарсенные с сайта данные
        получает на вход список ссылок объявлений и брэнд авто"""
    
    for i in range(len(links)):   
    
        time.sleep(0.3)
        response=requests.get(links[i])
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        new_list =[]
        auto = soup.find_all('span', class_ ='CardInfoRow__cell' )
        auto_list = list(auto)
    
        for j in range(len(auto_list)):
            new_list.append(auto_list[j].text)
        
    #Таможня отдельно
        try:
            customs = soup.find('li', class_ ='CardInfoRow CardInfoRow_customs')
            customs1 = list(customs) 
            customs_for_row = customs1[1].text
        except:
            customs_for_row = 'NaN'
    
    # Цена отдельно
        try:
            price = soup.find('span', class_ ='OfferPriceCaption__price')
            price1 = str(price)
            price_list = re.findall('\d+', price1)
            price_for_row = "".join(price_list)
        except:
            price_for_row = 'NaN'
    
    #Пробег отдельно
        try:
            
            kmage = soup.find('li', class_ ='CardInfoRow CardInfoRow_kmAge')
            kmage1 = str(kmage)
            kmage_list =  re.findall('\d+', kmage1)
            kmage_for_row = "".join(kmage_list)
        except:
            kmage_for_row = 'NaN'
    #Владение отдельно
        try:
            vladenie = soup.find('li', class_ ='CardInfoRow CardInfoRow_owningTime')
            vladenie1 = list(vladenie)
            vladenie_for_row = vladenie1[1].text
        except:
            vladenie_for_row = 'NaN'
        try:
            row = {'brand':brand,'productionDate':new_list[1],'bodyType':new_list[5],'color':new_list[7],'engineDisplacement':new_list[9][:5],
            'enginePower':new_list[9][7:11],'fuelType':new_list[9][-6:],'vehicleTransmission':new_list[13],
            'Привод':new_list[15],'Руль':new_list[17],'Владение':vladenie_for_row, 'Состояние':new_list[19],'Таможня':customs_for_row,'price':price_for_row,'kmage':kmage_for_row,
              'Владельцы':new_list[21][0],'car_url':links[i]}
            
            with open("parsing_finall.csv", "a",encoding='utf-8', newline="") as file:
                columns = ['brand','productionDate','bodyType','color','engineDisplacement',
            'enginePower','fuelType','vehicleTransmission',
            'Привод','Руль','Владение','Состояние','Таможня','price','kmage',
              'Владельцы','car_url']
                writer = csv.DictWriter(file, fieldnames=columns)
                writer.writerow(row)
        except:
            continue
            
            
           
    return

In [ ]:
model_all = ['bmw', 'volkswagen', 'nissan', 'mercedes', 'toyota', 'audi', 'mitsubishi', 'skoda', 
             'volvo', 'honda', 'infiniti', 'lexus']
for i in range(1,500):     # кол-во страниц для парсинга с авто.ру 
    for brand in model_all:  # марка авто
        res = requests.get(f'https://auto.ru/moskva/cars/{brand}/used/?page={i}')
        soup = BeautifulSoup(res.text, 'html.parser')
        container = soup.find_all('div', class_ ='ListingItem-module__thumb')
        #список ссылок объявлений на странице
        a = list(container)
        links = list(map(lambda x: x.find('a').get('href'),a))
        file_builder(links,brand)